# Lektion 13

In [ ]:
from sympy import *
init_printing()
import numpy as np

# Zufallsvariable

In [ ]:
from sympy.stats import *

In [ ]:
X = DiscreteUniform('X', range(16))
X

In [ ]:
E(X**2)

In [ ]:
variance(X**2)

In [ ]:
sample(X)

In [ ]:
l = []
for j in range(10000):
    l.append(sample(X))

# dasselbe in numpy

In [ ]:
np.random.choice(np.arange(16), 10)

In [ ]:
zv = np.random.choice(np.arange(16), 10000)

Was ist der Unterschied?

In [ ]:
x = S('x')
f = sqrt(1+sin(x)**4)
f

In [ ]:
fn = lambdify(x, f)  # "numpy" ist default, wenn es installiert ist

In [ ]:
fn(0)

In [ ]:
fn(zv)  

Was passiert hier?

In [ ]:
g = besselk(2*I, x) / f
g

In [ ]:
ddg = g.diff(x, 2).simplify()
ddg

In [ ]:
ddgn = lambdify(x, ddg)

In [ ]:
# ddgn(1)   # TypeError

In [ ]:
ddgn = lambdify(x, ddg, modules=['scipy', 'numpy', 'mpmath'])

In [ ]:
ddgn(1)

In [ ]:
# ddgn(zv)   # TypeError

`mpmath` beherrscht keine ufuncs

# Ausgabe in anderen Programmiersprachen

Gesetzt den Fall, Sie haben eine Fortran-Bibliothek, die komplexe Besselfunktionen beherrscht:

In [ ]:
print(fcode(f))

In [ ]:
print(fcode(ddg))

In [ ]:
fc = fcode(ddg, user_functions={'besselk': 'my_lib_Kv'}, assign_to='res', standard=95)
print(fc)

In [ ]:
cc = cxxcode(ddg)
print(cc)

# Pattern matching 

In [ ]:
x = Symbol('x')
w = Wild('w')   #  wildcard

In [ ]:
pattern = sin(w)

In [ ]:
a = sin(2*x+pi/4)
a

In [ ]:
m = a.match(pattern)
m

In [ ]:
b = 2*sin(w/2)*cos(w/2)
b

In [ ]:
b.subs(m)

In [ ]:
def expand_sin_x_halbe(term):
    x1 = Wild('w')
    pattern = sin(w)
    ersetzung = 2*sin(w/2)*cos(w/2)
    m = term.match(pattern)
    if m:
        return ersetzung.subs(m)
    else:
        return term

In [ ]:
expand_sin_x_halbe(sin(3*x))

In [ ]:
f = sin(3*x+19)
(expand_sin_x_halbe(f)-f).simplify()

In [ ]:
f = 3*sin(x)
expand_sin_x_halbe(f)

In [ ]:
type(f)

In [ ]:
f.args

In [ ]:
expand_sin_x_halbe(f.args[0]) * expand_sin_x_halbe(f.args[1])

In [ ]:
def expand_sin_x_halbe(ausdr):
    ausdr = S(ausdr)
    w = Wild('w')
    pattern = sin(w)
    ersetzung = 2*sin(w/2)*cos(w/2)
    m = ausdr.match(pattern)
    if m:
        res = ersetzung.subs(m)
    elif ausdr.is_Mul:
        res = 1
        for term in ausdr.args:
            res = res * expand_sin_x_halbe(term)
    elif ausdr.is_Add:
        res = 0
        for term in ausdr.args:
            res = res + expand_sin_x_halbe(term)
    else:
        res = ausdr
    return res

In [ ]:
b = 3*sin(x)/(1+sin(x/2)**2)
b

In [ ]:
expand_sin_x_halbe(b)

In [ ]:
x,y,z = symbols('x y z')
a = x + y + z

In [ ]:
a.args

In [ ]:
a.func

In [ ]:
a.func(*a.args)

In [ ]:
def expand_sin_x_halbe(ausdr):
    ausdr = S(ausdr)
    w = Wild('w')
    pattern = sin(w)
    ersetzung = 2*sin(w/2)*cos(w/2)
    m = ausdr.match(pattern)
    if m:
        res = ersetzung.subs(m)
    else:
        try:
            f = ausdr.func
        except AttributeError:
            res = ausdr
        args = ausdr.args
        if len(args) == 0:
            res = ausdr
        else:
            liste = []
            for l in args:
                liste.append(expand_sin_x_halbe(l))
            res = f(*liste)
    return res

In [ ]:
expand_sin_x_halbe(2*sin(x/2))

In [ ]:
expand_sin_x_halbe(b)

# Zurück zu den Besselfunktionen

In [ ]:
nu = S('nu')

In [ ]:
f = besselj(nu, x)
g = bessely(nu, x)

In [ ]:
M = Matrix([[f,g], [f.diff(x), g.diff(x)]])
M

In [ ]:
W = M.det().expand()
W

https://dlmf.nist.gov/10.6

In [ ]:
def bessel_forward(ausdr, bessel, nu):
    w1 = Wild('w1')
    w2 = Wild('w2')
    w3 = Wild('w3')
    pattern = bessel(w1, w2)
    if bessel == besselj or bessel == bessely:
        ersetzung = -bessel(nu+2, w2) + 2*(nu+1)/w2*bessel(nu+1, w2)    
    m = ausdr.match(pattern)
    if m and w1.subs(m) == nu:
        res = ersetzung.subs(m)
    else:
        try:
            f = ausdr.func
        except AttributeError:
            res = ausdr
        args = ausdr.args
        if len(args) == 0:
            res = ausdr
        else:
            liste = []
            for l in args:
                liste.append(bessel_forward(l, bessel, nu))
            res = f(*liste)
    return res

In [ ]:
def bessel_backward(ausdr, bessel, nu):
    w1 = Wild('w1')
    w2 = Wild('w2')
    w3 = Wild('w3')
    pattern = bessel(w1, w2)
    if bessel == besselj or bessel == bessely:
        ersetzung = -bessel(nu-2, w2) + 2*(nu-1)/w2*bessel(nu-1, w2)            
    m = ausdr.match(pattern)
    if m and w1.subs(m) == nu:
        res = ersetzung.subs(m)
    else:
        try:
            f = ausdr.func
        except AttributeError:
            res = ausdr
        args = ausdr.args
        if len(args) == 0:
            res = ausdr
        else:
            liste = []
            for l in args:
                liste.append(bessel_backward(l, bessel, nu))
            res = f(*liste)
    return res

In [ ]:
tmp = bessel_forward(W, bessely, nu-1).expand()
tmp

In [ ]:
tmp = bessel_forward(tmp, besselj, nu-1).expand()
tmp

In [ ]:
tmp = tmp.diff(x).expand()

In [ ]:
tmp = bessel_forward(tmp, bessely, nu-1).expand()
tmp = bessel_forward(tmp, besselj, nu-1).expand()
tmp

In [ ]:
tmp = bessel_backward(tmp, bessely, nu+2).expand()
tmp = bessel_backward(tmp, besselj, nu+2).expand()
tmp

 Also erfüllt die Wronskische die Dgl $W'=-\frac Wx$

In [ ]:
y = Function('y')
dgl = Eq(y(x).diff(), -y(x)/x)
dgl

In [ ]:
dsolve(dgl)